In [1]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

Define hyperparameters

In [2]:
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 5

MAX_SEQ_LENGTH = 40
NUM_FEATURES = 2048

INDEX = 15

In [3]:
train_df = pd.read_csv("video list/train.csv")
test_df = pd.read_csv("video list/test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

#train_df.sample(10)

Total videos for training: 80
Total videos for testing: 20


In [4]:
train_data = np.load(f"Index{INDEX}/_features/train_data.npy")
train_labels = np.load(f"Index{INDEX}/_features/train_labels.npy")

test_data = np.load(f"Index{INDEX}/_features/test_data.npy")
test_labels = np.load(f"Index{INDEX}/_features/test_labels.npy")

print(f"Frame features in train set: {train_data.shape}")
print(f"Frame label in train set: {train_labels.shape}")

print(f"Frame features in test set: {test_data.shape}")
print(f"Frame label in test set: {test_labels.shape}")

Frame features in train set: (580, 40, 2048)
Frame label in train set: (580, 1)
Frame features in test set: (141, 40, 2048)
Frame label in test set: (141, 1)


Modificar Datos

In [33]:
column = train_labels[:, 0]
column[column < 3] = 0
column[column >= 3] = 1

column = test_labels[:, 0]
column[column < 3] = 0
column[column >= 3] = 1

print(f"Frame features in train set: {train_labels.shape}")
print(f"Frame features in test set: {test_labels.shape}")

Frame features in train set: (580, 1)
Frame features in test set: (141, 1)


Contar Datos

In [5]:
import numpy as np

def Contar(tensor):
    unique_values, counts = np.unique(tensor, return_counts=True)
    totalSum = np.sum(counts)
    for value, count in zip(unique_values, counts):
        print(f"Número {value}: {count} veces, {(count / totalSum) * 100}")

print(f"Train")
Contar(train_labels[:, 0])

print(f"Test")
Contar(test_labels[:, 0])

Train
Número 0.0: 38 veces, 6.551724137931035
Número 1.0: 77 veces, 13.275862068965516
Número 2.0: 29 veces, 5.0
Número 3.0: 436 veces, 75.17241379310344
Test
Número 0.0: 14 veces, 9.929078014184398
Número 1.0: 26 veces, 18.439716312056735
Número 2.0: 13 veces, 9.219858156028367
Número 3.0: 88 veces, 62.4113475177305


In [6]:
#train__data = np.reshape(train_data, (a * b, c))
def Reshape(data, labels):
    a, b, c = data.shape
    print()
    print("Reshape")
    print(f"a {a}")
    print(f"b {b}")
    print(f"c {c}")

    data_ = np.zeros((a * b, c))
    labels_ = np.zeros((a * b, 1))

    for i in range(a):
        for j in range(b):
            data_[i * j, : ] = data[i, j, :]
            labels_[i * j, :] = labels[i, :]

    print(data.shape)
    print(labels.shape)
    print()
    print(data_.shape)
    print(labels_.shape)

    return data_, labels_

train__data, train__labels = Reshape(train_data, train_labels)
test__data, test__labels = Reshape(test_data, test_labels)


Reshape
a 580
b 40
c 2048
(580, 40, 2048)
(580, 1)

(23200, 2048)
(23200, 1)

Reshape
a 141
b 40
c 2048
(141, 40, 2048)
(141, 1)

(5640, 2048)
(5640, 1)


The sequence model

In [9]:
from keras.models import Sequential
from keras.layers import Dense

def get_model():
    model = keras.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(NUM_FEATURES,)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(len(np.unique(train_df["Target"])), activation='softmax')
    ])

    # Compilar el modelo
    model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

    # Imprimir un resumen del modelo
    #model.summary()
    return model

# Utility for running experiments.
def run_experiment():
    filepath = f"Index{INDEX}/_model/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_model()
    #history = seq_model.fit( train__data, train__labels, validation_split=0.3, epochs=EPOCHS, callbacks=[checkpoint],)

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate(test__data, test__labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    return seq_model

print(f"Index{INDEX}")
print(test_data.shape)
sequence_model = run_experiment()

Index15
(141, 40, 2048)
177/177 [==============================] - 1s 2ms/step - loss: 0.9990 - accuracy: 0.8473
Test accuracy: 84.73%
0.9989914298057556


In [10]:
import numpy as np
from sklearn.metrics import classification_report

# Obtener las predicciones del modelo
y_pred = sequence_model.predict(test__data)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calcular el test accuracy general
_, test_accuracy = sequence_model.evaluate(test__data, test__labels, verbose=0)
print('Test Accuracy: {:.2f}%'.format(test_accuracy * 100))

# Obtener el classification report
report = classification_report(test__labels, y_pred_classes)
print('Classification Report:\n', report)

Train
[0. 1. 2. 3.]
[ 38  77  29 436]
580


Inference

In [37]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)

KeyError: 'video_name'